We begin by importing all the necssary packages and modules. Next our task is to build the Machine_Replacement environment. For doing that, we create a class named as Machine_Replacement which accepts the number_of_states(nS), number_of_actions(nA) and replacement_cost(rep_cost) as input and generate the environment. Later we just need to cal the function gen_probability() and gen_expected_reward_function() to get the Probability distribution matrix and Reward matrix

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import numpy as np
import pandas as pd
import pickle
from itertools import product
import multiprocessing as mp
mp.set_start_method('spawn',True)
torch.multiprocessing.set_start_method('spawn',True)
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Machine_Replacement:
    def __init__(self,rep_cost=0.7,nS=6,nA=2):
        self.nS = nS;
        self.nA = nA;
        self.cost = np.linspace(0.1, 0.99,nS);
        self.rep_cost = rep_cost;
    def gen_probability(self):
        self.P = np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            for j in range(self.nS):
                if(i<=j):
                    self.P[0,i,j]=(i+1)*(j+1);
                else:
                    continue;
            self.P[0,i,:]=self.P[0,i,:]/np.sum(self.P[0,i,:])
            self.P[1,i,0]=1;
        return self.P;
    def gen_reward(self):
        self.R=np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            self.R[0,i,:] = self.cost[i];
            self.R[1,i,0] = self.rep_cost+self.cost[0];
        return self.R;
    def gen_expected_reward(self):
        self.R = np.zeros((self.nA,self.nS));
        for i in range(self.nS):
            self.R[0,i] = self.cost[i];
            self.R[1,i] = self.rep_cost + self.cost[0];
        return self.R;

Next we create another class specifying the hyperparameters that might be required in our algorithm. Later when required we can just call ret_hyperparameters() to get the hyperparameters.

In [14]:
class get_hyperparameters:
    def __init__(self):
        self.T = 50;
        self.runs = 5;
        self.lr = 0.1;
        self.batch_size = 25;
        self.start = 0;
        self.nS = 4;
        self.nA = 2;
        self.rep_cost = 0.7
        self.alpha = 0.2
        self.gamma = 0.95
    
    def ret_hyperparameters(self):
        return (self.T,self.runs,self.lr,self.batch_size,self.start,self.nS,self.nA,self.rep_cost,self.alpha,self.gamma)

Let us now define the pytorch model. So for doing that, we create a class named weights. There are 3 parameters, input_size which defines the number of perceptrons in the input layer. input_size = number of states(nS). The output size is the number of perceptrons in the output_layer. output_size = 1(which gives us the state distribution ratio for a particular state).

Later when we want to update or find the state distribution ratio of any state, just pass that state to the forward(). First that particular state is converted into one_hot vector and then fed to the network. Finally the network returns the output value as the ratio of state distribution.

In [15]:
class weights(nn.Module):
    def __init__(self,input_size,output_size,hidden_size = 0):
        super(weights,self).__init__()
        self.input_size = input_size;
        self.hidden_size = hidden_size;
        self.output_size = output_size;
        if(hidden_size!=0):
            self.linear1 = nn.Linear(self.input_size, self.hidden_size, bias=False)
            self.linear2 = nn.Linear(self.hidden_size, self.output_size, bias=False)
        else:
            self.linear1 = nn.Linear(self.input_size, self.output_size, bias=False)
    '''
        forward(): We accept a state 's' as input. Then we convert this into one hot encoding which is accomplished by first two lines.
        Further we convert this one_hot vector 's' into pytorch tensor and then pass it through the network to obtain a output which is returned 
    '''
    def forward(self,state):
        s = np.zeros(self.input_size);
        #print(state,end='===>');
        s[state] = 1;
        state = torch.FloatTensor(s).to(device)
        #print(state);
        if(self.hidden_size == 0):
            output = torch.exp(self.linear1(state)) #To ensure that the outputs are always positive. giving Relu will cause problems.
        else:
            output = torch.exp(self.linear2(torch.exp(self.linear1(state))));
        return output

Now it is evident that the network written above will give me the ratio of state distribution. But the requirement is state distribution of the target_policy. So for doing that we need to to follow the below equation

state_distribution_of_target_policy = Normalize(state_distribution_ratio_obtained_from_network * behaviour_policy_state_distribution).

Now to get the target_policy_state_distribution, we need to obtain the behaviour_policy_state_distribution. To find that we use the class below. Now using the 

In [16]:
class beh_pol_sd:
    def __init__(self,P,policy,nS,nA):
        self.P = P
        self.policy = policy
        self.nS = nS;
        self.nA = nA;
    
    def onehot(self):
        pol = np.zeros((self.nS,self.nA));
        for i in range(self.nS):
            pol[i][int(self.policy[i])]=1;
        return pol;
    def find_transition_matrix(self,onehot_encode=1):
        if(onehot_encode==1):
            self.policy = self.onehot()
        T_s_s_next = np.zeros((self.nS,self.nS));
        for s in range(self.nS):
            for s_next in range(self.nS):
                for a in range(self.nA):
                    #print(s,s_next,a);
                    #print(T[a,s,s_next]);
                    T_s_s_next[s,s_next]+=self.P[a,s,s_next]*self.policy[s,a];
        return T_s_s_next;
    def state_distribution_simulated(self,onehot_encode=1):
        P_policy = self.find_transition_matrix(onehot_encode)
        #print(P_policy);
        P_dash = np.append(P_policy - np.eye(self.nS),np.ones((self.nS,1)),axis=1);
        #print(P_dash);
        P_last = np.linalg.pinv(np.transpose(P_dash))[:,-1]
        return P_last;

In [17]:
def one_hot(target_policy,nS,nA):
    one_hot_tp = [];
    for i in range(len(target_policy)):
        policy = target_policy[i];
        print(policy);
        tp=np.zeros((nS,nA));
        for j in range(nS):
            tp[j][policy[j]] = 1;
        one_hot_tp.append(tp);
    return np.array(one_hot_tp);

Now that we are all ready got our ingedients, let us define a separate class where we will define the average_case technique to find the state_distribution_ratio. Any object of the average_case class must possess the environment details such as number of states(nS) and number of actions(nA). The behaviour_policy, learning_rate,list_of_beta_values, a weight_object to refer to the weights class. The batch_size and the optimizer to be used(Adam_optimzer). 

Since in this variant, we will be having only 1 Neural Network and update the network for the sample that is obtained from the theta values. For this reason, we need to store all the beta values. Hence, we define our first function
1) find_beta(): This function finds all the beta(importance sampling) values. So beta values = target_policy[s,a]/behaviour_policy[s,a]. Now there are nPOL policies so for each policies we need to find the beta values for the ith target_policy, beta value will be beta[i,s,a]=target_policy[i,s,a]/behaviour_policy[s,a]. Now it is certain that a(action) can be {0,1} only. So instead of creating a new loop, we manually define two lines once for a=0 when beta value becomes beta[i,s,0]=target_policy[i,s,0]/behaviour_policy[s,0] and beta[i,s,1] = target_policy[i,s,1]/behaviour_policy[s,1]. Finally return all the beta values.

2) set_batch(): This function is used to set a data batch which is sampled from the behaviour_policy. Now the batch is set to be used for updating the state_distribution_ratio.

3) get_batch(): This function is used to get a random batch of 50 samples from the set data to be used to update our state_distribution_ratio. We actually create a batch of 50 data samples 10000 times in order to reach to a good value of state_distribution_ratio(Like in Linear regression we use a batch for several times until our gradient converges)

4) get_w(): This function is used to find the numerator and denominator of the loss function as mentioned in the paper 'Breaking the curse of horizon'. Now for finding the numerator paarameter pair = 0. To find the denominator pair = 1. Now it is observed that the denominator value easily goes to 0. So, to avoid divide by zero error, we add a small noise value of 0.000000001. This makes sure that the denominator value never goes to zero.

5) get_state_distribution_ratio(): This function uses the set data in the self.set_data() to get batches of size 50. Then calculate the loss using the equation mentioned in the paper 'Breaking the curse of horizon'. We use this calculated loss to update our weights of the Neural network by using Adam optimizer.

In [18]:
class average_case_version_2:
  def __init__(self,nS,nA,behaviour_policy,state,lr,target_policy,batch_size,data_used):
    self.nS = nS;
    self.nPOL = nS
    self.nA = nA;
    self.behaviour_policy = behaviour_policy;
    self.lr = lr;
    self.beta = self.find_beta(target_policy,behaviour_policy)
    self.W_loss = 0
    self.weight_obj = weights(nS,1).to(device);
    self.batch_size = batch_size
    self.optimizerW = optim.Adam(self.weight_obj.parameters(),lr = self.lr);
    self.loss = [];
    self.Z = [];
    self.data_used = data_used
  def find_beta(self,target_policy,behaviour_policy):
    beta = np.zeros((self.nPOL,self.nS,self.nA))
    for i in range(self.nPOL):
      for s in range(self.nS):
        beta[i][s][0] = target_policy[i,s,0]/behaviour_policy[s,0];
        beta[i,s,1] = target_policy[i,s,1]/behaviour_policy[s,1];
    return beta;
  def set_batch(self,data):
        self.data = data;
        self.T = len(data);
  def get_batch(self):
      if(self.T<=50):
          return self.data
      else:
          i = 1;
          j = np.random.choice(self.T);
          batch = [];
          while(i<=self.batch_size):
              if(np.random.random()<=0.5):
                  batch.append([self.data[j][0],self.data[j][1],self.data[j][2]])
                  j = (j+1)%self.T;
                  i+=1;
          return batch; 
  def get_w(self,data,weight_obj,m,pair=0):
        if(pair == 1):
            Z_w_state = 0;
            for i in range(len(data)):
                val = weight_obj(data[i][0]);
                #print(val);
                Z_w_state+=val;
            #print(Z_w_state.detach().numpy()[0]/self.batch_size);
            Z_w_state = Z_w_state.cpu().detach().numpy()[0]/self.batch_size;
            if(Z_w_state<0.00000000000005):
                Z_w_state+=0.000000000001;
            return Z_w_state;
        else:
            state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2 = list(),list(),list(),list(),list(),list(),list(),list();
            K = list();
            for i in range(len(data)):
                sample1 = data[i][0];
                sample2 = data[i][1];
                state1.append(sample1[0]);
                #print(sample1);
                w_state1.append(weight_obj(sample1[0]));
                w_next_state1.append(weight_obj(sample1[2]));
                state2.append(sample2[0]);
                w_state2.append(weight_obj(sample2[0]));
                w_next_state2.append(weight_obj(sample2[2]));
                #beta1.append(self.target_policy[sample1[0],sample1[1]]/self.behaviour_policy[sample1[0],sample1[1]]);
                beta1.append(self.beta[self.selected_policy,sample1[0],sample1[1]]);
                #beta2.append(self.target_policy[sample2[0],sample2[1]]/self.behaviour_policy[sample2[0],sample2[1]]);
                beta2.append(self.beta[self.selected_policy,sample2[0],sample2[1]])
                K.append(sample1[2]==sample2[2]);
            return (state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K);
    
  def get_state_distribution_ratio(self,selected_policy,run,t):
        batch = self.get_batch();
        eps = 0.04;
        self.data_used[run] =self.data_used[run]+batch;
        self.selected_policy = selected_policy
        pairs = list(product(batch,repeat=2));
        self.loss_episode = [];
        for _ in range(100):
            batch = self.get_batch();
            state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K = self.get_w(pairs, self.weight_obj, len(batch));
            Z_w_state = self.get_w(batch, self.weight_obj, len(batch),1);
            self.w_loss = 0
            for i in range(len(state1)):
                self.w_loss+=(beta1[i]*(w_state1[i]/Z_w_state) - (w_next_state1[i]/Z_w_state))*(beta2[i]*(w_state2[i]/Z_w_state)-(w_next_state2[i]/Z_w_state))*K[i];
            self.w_loss/=(2*self.batch_size);
            self.optimizerW.zero_grad();
            self.w_loss.backward();
            self.optimizerW.step();
            self.optimizerW.zero_grad();
            self.Z.append(Z_w_state)
        self.loss.append(self.w_loss.cpu().detach().numpy()[0]);
        state_dist=[];
        for i in range(self.nS):
            w_state = self.weight_obj(i);
            w_state = w_state.cpu().detach().numpy()[0];
            state_dist.append(w_state);
        return np.array(state_dist);

Instead of sampling state, action, next_state values on the go, we do it before hand and store it in a list named 'data'. So, that when required we can simply pass this data and save some time by before hand sampling the data.

In [19]:
def simulate_episode(T,state,behaviour_policy,P,batch_size):
  #global P,behaviour_policy,batch_size;
  data={};temp=[];
  for t in range(1,T+1):
    action = np.argmax(np.random.multinomial(1,behaviour_policy[state,:]))
    next_state = np.argmax(np.random.multinomial(1,P[action,state,:]));
    state = next_state;
    temp.append([state,action,next_state]);
    if(t%batch_size==0):
      data[int(t/batch_size)-1]=temp[:];
  return data;

For our help we create a softmax function.

In [20]:
def softmax(theta):
  theta = np.exp(theta);
  sum = np.sum(theta)
  return theta/sum;

We create a function naemd as preprocessing() which is responsible to perform the theta update. After every val=50 times, our learning rate is divided by 10. So if we start with lr = 1. After 50 instances, lr = 0.1. After 100 steps lr = 0.01 and so on...
Now at each instant, we found the softmax of the theta values. Now from the given probability values we sample a policy that will be updated next. 

In [21]:
def processing(nPOL,run,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val,val2,behaviour_policy_state_distribution,beta,batch_size,alpha):
    lr = 1
    nS = nPOL;
    theta = np.ones(nPOL);
    c=0;
    rew = np.zeros(nPOL);
    S = np.ones(nPOL);
    F = np.ones(nPOL);
    n = np.ones(nPOL);
    for t in tqdm(range(1,T_update+1)):
      sampled_policy = np.argmin([np.random.beta(S[j],F[j]) for j in range(nPOL)])
      #policy_selected[t,run]=selected_policy;
      policy_sampled[t-1,run] = sampled_policy;
      #loss[t,run] = w_obj.w_loss.cpu().detach().numpy()[0];
      w_obj.set_batch(data[t-1]);
      sd = w_obj.get_state_distribution_ratio(sampled_policy,run,t-1);
      sd = sd * behaviour_policy_state_distribution;
      sd = sd/np.sum(sd)
      rho_i = sum([sd[s]*R[target_policy[sampled_policy,s],s] for s in range(nS)]);
      rew[sampled_policy] = (1-alpha)*rew[sampled_policy] + rho_i
      S[sampled_policy]+=rew[sampled_policy];
      F[sampled_policy]=F[sampled_policy]+t-rew[sampled_policy]
      estimated_value[t-1,run] = rho_i;
    print(data)

Finally call the hyperparameters, create the environment and call the pre-processing function.

In [22]:
if __name__ =='__main__':
    T,runs,lr,batch_size,state,nS,nA,rep_cost,alpha,gamma = get_hyperparameters().ret_hyperparameters();
    print(nS,nA);
    nPOL = nS;
    T_update = int(T/batch_size);
    mr_obj = Machine_Replacement(rep_cost,nS,nA);
    P,R = mr_obj.gen_probability(),mr_obj.gen_expected_reward()
    theta = np.ones(nPOL);
    behaviour_policy = np.ones((nS,nA))*0.5;
    print(behaviour_policy,P);
    behaviour_policy_state_distribution = beh_pol_sd(P,behaviour_policy,nS,nA).state_distribution_simulated(0);
    data = simulate_episode(T,state,behaviour_policy,P,batch_size);
    with open('data_used','wb') as f:
      pickle.dump(data,f);
    target_policy = np.ones((nPOL,nS),dtype = np.int8)
    #data_dict={0:[],1:[],2:[],3:[],4:[]};
    val2 = []
    for i in range(nPOL-1,0,-1):
        target_policy[nPOL-i-1][0:i] = 0;
    for t in target_policy:
        val2.append(sum([R[t[s],s] for s in range(nS)]))
    val2 = np.array(val2);
    one_hot_target_policy = one_hot(target_policy,nS,nA)
    w_obj = average_case_version_2(nS,nA,behaviour_policy,state,lr,one_hot_target_policy,batch_size,data_dict);
    '''for i in range(nPOL):
      w_obj[i].set_target_policy(one_hot_target_policy[i]);'''
    policy_selected = np.zeros((T_update,runs))
    policy_sampled = np.zeros((T_update,runs))
    theta_change = []
    estimated_value = np.zeros((T_update,runs))
    loss = np.zeros((T_update,runs))
    val = 50;
    lr = 1;
    beta = 1;
    #######################################################################################
    '''mp.set_start_method('spawn')
    p1 = mp.Process(target = processing ,args=(nPOL,0,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p2 = mp.Process(target = processing ,args=(nPOL,1,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p3 = mp.Process(target = processing ,args=(nPOL,2,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p4 = mp.Process(target = processing ,args=(nPOL,3,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p5 = mp.Process(target = processing ,args=(nPOL,4,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    p1.start();
    p2.start();
    p3.start();
    p4.start();
    p5.start();
    p1.join();
    p2.join();
    p3.join();
    p4.join();
    p5.join();'''
    #######################################################################################
    for run in tqdm(range(runs)):
        processing(nPOL,run,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val,val2,behaviour_policy_state_distribution,beta,batch_size,alpha)
        print("One run completed");
    #######################################################################################
    #pd.DataFrame(policy_selected).to_excel("Policy_selection.xlsx");
    #pd.DataFrame(np.array(theta_change)).to_excel("Theta_values_non_estimate.xlsx");
    pd.DataFrame(policy_sampled).to_excel("Policy_sampling_Thompson_Sampling_variant_2.xlsx");
    pd.DataFrame(data).to_excel("Data_Used_Thompson_Sampling_variant_2.xlsx");
    pd.DataFrame(estimated_value).to_excel("Estimated_Value_functions_Thompson_Sampling_variant_2.xlsx");

4 2
[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]] [[[0.1        0.2        0.3        0.4       ]
  [0.         0.22222222 0.33333333 0.44444444]
  [0.         0.         0.42857143 0.57142857]
  [0.         0.         0.         1.        ]]

 [[1.         0.         0.         0.        ]
  [1.         0.         0.         0.        ]
  [1.         0.         0.         0.        ]
  [1.         0.         0.         0.        ]]]
[0 0 0 1]
[0 0 1 1]
[0 1 1 1]
[1 1 1 1]


 20%|████████▊                                   | 1/5 [03:43<14:54, 223.73s/it]

{0: [[2, 0, 2], [0, 1, 0], [2, 0, 2], [0, 1, 0], [1, 0, 1], [0, 1, 0], [3, 0, 3], [3, 0, 3], [3, 0, 3], [3, 0, 3], [3, 0, 3], [0, 1, 0], [0, 1, 0], [2, 0, 2], [3, 0, 3], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [3, 0, 3], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]], 1: [[2, 0, 2], [0, 1, 0], [2, 0, 2], [0, 1, 0], [1, 0, 1], [0, 1, 0], [3, 0, 3], [3, 0, 3], [3, 0, 3], [3, 0, 3], [3, 0, 3], [0, 1, 0], [0, 1, 0], [2, 0, 2], [3, 0, 3], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [3, 0, 3], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 1], [0, 1, 0], [3, 0, 3], [3, 0, 3], [0, 1, 0], [2, 0, 2], [3, 0, 3], [0, 1, 0], [3, 0, 3], [0, 1, 0], [2, 0, 2], [2, 0, 2], [3, 0, 3], [3, 0, 3], [0, 1, 0], [1, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [2, 0, 2], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]]}
One run completed



 20%|████████▊                                   | 1/5 [05:20<21:20, 320.16s/it]


KeyboardInterrupt: 